In [1]:
import pandas as pd
import numpy as np
import ta
import pickle
from tvDatafeed import TvDatafeed, Interval
from backtesting import Backtest, Strategy
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

c:\Users\visha\AppData\Local\Programs\Python\Python313\Lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [5]:
# weighted data from NIFTY

tv = TvDatafeed()
nifty_index_data = tv.get_hist(symbol='NIFTY',exchange='NSE',interval=Interval.in_30_minute,n_bars=2000)
print(nifty_index_data)

                        symbol      open      high       low     close  \
datetime                                                                 
2025-04-17 11:15:00  NSE:NIFTY  23440.95  23615.45  23439.35  23596.50   
2025-04-17 11:45:00  NSE:NIFTY  23598.85  23648.05  23567.40  23634.45   
2025-04-17 12:15:00  NSE:NIFTY  23634.25  23717.05  23623.00  23712.35   
2025-04-17 12:45:00  NSE:NIFTY  23712.05  23789.25  23703.30  23788.65   
2025-04-17 13:15:00  NSE:NIFTY  23787.85  23819.15  23761.70  23816.45   
...                        ...       ...       ...       ...       ...   
2025-11-28 13:15:00  NSE:NIFTY  26217.45  26234.90  26203.90  26217.70   
2025-11-28 13:45:00  NSE:NIFTY  26217.00  26218.10  26194.00  26200.75   
2025-11-28 14:15:00  NSE:NIFTY  26200.95  26222.45  26194.90  26205.55   
2025-11-28 14:45:00  NSE:NIFTY  26205.30  26214.40  26196.15  26197.55   
2025-11-28 15:15:00  NSE:NIFTY  26197.45  26213.65  26191.90  26204.55   

                         volume  
dat

In [6]:
# todo: automate fetching this

NIFTY_500_SYMBOLS = [
    '360ONE',
    '3MINDIA',
    'ABB',
    'ACC',
    'ACMESOLAR',
    'AIAENG',
    'APLAPOLLO',
    'AUBANK',
    'AWL',
    'AADHARHFC',
    'AARTIIND',
    'AAVAS',
    'ABBOTINDIA',
    'ACE',
    'ADANIENSOL',
    'ADANIENT',
    'ADANIGREEN',
    'ADANIPORTS',
    'ADANIPOWER',
    'ATGL',
    'ABCAPITAL',
    'ABFRL',
    'ABLBL',
    'ABREL',
    'ABSLAMC',
    'AEGISLOG',
    'AEGISVOPAK',
    'AFCONS',
    'AFFLE',
    'AJANTPHARM',
    'AKUMS',
    'AKZOINDIA',
    'APLLTD',
    'ALKEM',
    'ALKYLAMINE',
    'ALOKINDS',
    'ARE&M',
    'AMBER',
    'AMBUJACEM',
    'ANANDRATHI',
    'ANANTRAJ',
    'ANGELONE',
    'APARINDS',
    'APOLLOHOSP',
    'APOLLOTYRE',
    'APTUS',
    'ASAHIINDIA',
    'ASHOKLEY',
    'ASIANPAINT',
    'ASTERDM',
    'ASTRAZEN',
    'ASTRAL',
    'ATHERENERG',
    'ATUL',
    'AUROPHARMA',
    'AIIL',
    'DMART',
    'AXISBANK',
    'BASF',
    'BEML',
    'BLS',
    'BSE',
    'BAJAJ-AUTO',
    'BAJFINANCE',
    'BAJAJFINSV',
    'BAJAJHLDNG',
    'BAJAJHFL',
    'BALKRISIND',
    'BALRAMCHIN',
    'BANDHANBNK',
    'BANKBARODA',
    'BANKINDIA',
    'MAHABANK',
    'BATAINDIA',
    'BAYERCROP',
    'BERGEPAINT',
    'BDL',
    'BEL',
    'BHARATFORG',
    'BHEL',
    'BPCL',
    'BHARTIARTL',
    'BHARTIHEXA',
    'BIKAJI',
    'BIOCON',
    'BSOFT',
    'BLUEDART',
    'BLUEJET',
    'BLUESTARCO',
    'BBTC',
    'BOSCHLTD',
    'FIRSTCRY',
    'BRIGADE',
    'BRITANNIA',
    'MAPMYINDIA',
    'CCL',
    'CESC',
    'CGPOWER',
    'CRISIL',
    'CAMPUS',
    'CANFINHOME',
    'CANBK',
    'CAPLIPOINT',
    'CGCL',
    'CARBORUNIV',
    'CASTROLIND',
    'CEATLTD',
    'CENTRALBK',
    'CDSL',
    'CENTURYPLY',
    'CERA',
    'CHALET',
    'CHAMBLFERT',
    'CHENNPETRO',
    'CHOICEIN',
    'CHOLAHLDNG',
    'CHOLAFIN',
    'CIPLA',
    'CUB',
    'CLEAN',
    'COALINDIA',
    'COCHINSHIP',
    'COFORGE',
    'COHANCE',
    'COLPAL',
    'CAMS',
    'CONCORDBIO',
    'CONCOR',
    'COROMANDEL',
    'CRAFTSMAN',
    'CREDITACC',
    'CROMPTON',
    'CUMMINSIND',
    'CYIENT',
    'DCMSHRIRAM',
    'DLF',
    'DOMS',
    'DABUR',
    'DALBHARAT',
    'DATAPATTNS',
    'DEEPAKFERT',
    'DEEPAKNTR',
    'DELHIVERY',
    'DEVYANI',
    'DIVISLAB',
    'DIXON',
    'AGARWALEYE',
    'LALPATHLAB',
    'DRREDDY',
    'DUMMYSKFIN',
    'EIDPARRY',
    'EIHOTEL',
    'EICHERMOT',
    'ELECON',
    'ELGIEQUIP',
    'EMAMILTD',
    'EMCURE',
    'ENDURANCE',
    'ENGINERSIN',
    'ERIS',
    'ESCORTS',
    'ETERNAL',
    'EXIDEIND',
    'NYKAA',
    'FEDERALBNK',
    'FACT',
    'FINCABLES',
    'FINPIPE',
    'FSL',
    'FIVESTAR',
    'FORCEMOT',
    'FORTIS',
    'GAIL',
    'GVT&D',
    'GMRAIRPORT',
    'GRSE',
    'GICRE',
    'GILLETTE',
    'GLAND',
    'GLAXO',
    'GLENMARK',
    'MEDANTA',
    'GODIGIT',
    'GPIL',
    'GODFRYPHLP',
    'GODREJAGRO',
    'GODREJCP',
    'GODREJIND',
    'GODREJPROP',
    'GRANULES',
    'GRAPHITE',
    'GRASIM',
    'GRAVITA',
    'GESHIP',
    'FLUOROCHEM',
    'GUJGASLTD',
    'GMDCLTD',
    'GSPL',
    'HEG',
    'HBLENGINE',
    'HCLTECH',
    'HDFCAMC',
    'HDFCBANK',
    'HDFCLIFE',
    'HFCL',
    'HAPPSTMNDS',
    'HAVELLS',
    'HEROMOTOCO',
    'HEXT',
    'HSCL',
    'HINDALCO',
    'HAL',
    'HINDCOPPER',
    'HINDPETRO',
    'HINDUNILVR',
    'HINDZINC',
    'POWERINDIA',
    'HOMEFIRST',
    'HONASA',
    'HONAUT',
    'HUDCO',
    'HYUNDAI',
    'ICICIBANK',
    'ICICIGI',
    'ICICIPRULI',
    'IDBI',
    'IDFCFIRSTB',
    'IFCI',
    'IIFL',
    'INOXINDIA',
    'IRB',
    'IRCON',
    'ITCHOTELS',
    'ITC',
    'ITI',
    'INDGN',
    'INDIACEM',
    'INDIAMART',
    'INDIANB',
    'IEX',
    'INDHOTEL',
    'IOC',
    'IOB',
    'IRCTC',
    'IRFC',
    'IREDA',
    'IGL',
    'INDUSTOWER',
    'INDUSINDBK',
    'NAUKRI',
    'INFY',
    'INOXWIND',
    'INTELLECT',
    'INDIGO',
    'IGIL',
    'IKS',
    'IPCALAB',
    'JBCHEPHARM',
    'JKCEMENT',
    'JBMA',
    'JKTYRE',
    'JMFINANCIL',
    'JSWENERGY',
    'JSWINFRA',
    'JSWSTEEL',
    'JPPOWER',
    'J&KBANK',
    'JINDALSAW',
    'JSL',
    'JINDALSTEL',
    'JIOFIN',
    'JUBLFOOD',
    'JUBLINGREA',
    'JUBLPHARMA',
    'JWL',
    'JYOTHYLAB',
    'JYOTICNC',
    'KPRMILL',
    'KEI',
    'KPITTECH',
    'KSB',
    'KAJARIACER',
    'KPIL',
    'KALYANKJIL',
    'KARURVYSYA',
    'KAYNES',
    'KEC',
    'KFINTECH',
    'KIRLOSBROS',
    'KIRLOSENG',
    'KOTAKBANK',
    'KIMS',
    'LTF',
    'LTTS',
    'LICHSGFIN',
    'LTFOODS',
    'LTIM',
    'LT',
    'LATENTVIEW',
    'LAURUSLABS',
    'THELEELA',
    'LEMONTREE',
    'LICI',
    'LINDEINDIA',
    'LLOYDSME',
    'LODHA',
    'LUPIN',
    'MMTC',
    'MRF',
    'MGL',
    'MAHSCOOTER',
    'MAHSEAMLES',
    'M&MFIN',
    'M&M',
    'MANAPPURAM',
    'MRPL',
    'MANKIND',
    'MARICO',
    'MARUTI',
    'MFSL',
    'MAXHEALTH',
    'MAZDOCK',
    'METROPOLIS',
    'MINDACORP',
    'MSUMI',
    'MOTILALOFS',
    'MPHASIS',
    'MCX',
    'MUTHOOTFIN',
    'NATCOPHARM',
    'NBCC',
    'NCC',
    'NHPC',
    'NLCINDIA',
    'NMDC',
    'NSLNISP',
    'NTPCGREEN',
    'NTPC',
    'NH',
    'NATIONALUM',
    'NAVA',
    'NAVINFLUOR',
    'NESTLEIND',
    'NETWEB',
    'NEULANDLAB',
    'NEWGEN',
    'NAM-INDIA',
    'NIVABUPA',
    'NUVAMA',
    'NUVOCO',
    'OBEROIRLTY',
    'ONGC',
    'OIL',
    'OLAELEC',
    'OLECTRA',
    'PAYTM',
    'ONESOURCE',
    'OFSS',
    'POLICYBZR',
    'PCBL',
    'PGEL',
    'PIIND',
    'PNBHOUSING',
    'PTCIL',
    'PVRINOX',
    'PAGEIND',
    'PATANJALI',
    'PERSISTENT',
    'PETRONET',
    'PFIZER',
    'PHOENIXLTD',
    'PIDILITIND',
    'PPLPHARMA',
    'POLYMED',
    'POLYCAB',
    'POONAWALLA',
    'PFC',
    'POWERGRID',
    'PRAJIND',
    'PREMIERENE',
    'PRESTIGE',
    'PGHH',
    'PNB',
    'RRKABEL',
    'RBLBANK',
    'RECLTD',
    'RHIM',
    'RITES',
    'RADICO',
    'RVNL',
    'RAILTEL',
    'RAINBOW',
    'RKFORGE',
    'RCF',
    'REDINGTON',
    'RELIANCE',
    'RELINFRA',
    'RPOWER',
    'SBFC',
    'SBICARD',
    'SBILIFE',
    'SJVN',
    'SKFINDIA',
    'SRF',
    'SAGILITY',
    'SAILIFE',
    'SAMMAANCAP',
    'MOTHERSON',
    'SAPPHIRE',
    'SARDAEN',
    'SAREGAMA',
    'SCHAEFFLER',
    'SCHNEIDER',
    'SCI',
    'SHREECEM',
    'SHRIRAMFIN',
    'SHYAMMETL',
    'ENRIN',
    'SIEMENS',
    'SIGNATURE',
    'SOBHA',
    'SOLARINDS',
    'SONACOMS',
    'SONATSOFTW',
    'STARHEALTH',
    'SBIN',
    'SAIL',
    'SUMICHEM',
    'SUNPHARMA',
    'SUNTV',
    'SUNDARMFIN',
    'SUNDRMFAST',
    'SUPREMEIND',
    'SUZLON',
    'SWANCORP',
    'SWIGGY',
    'SYNGENE',
    'SYRMA',
    'TBOTEK',
    'TVSMOTOR',
    'TATACHEM',
    'TATACOMM',
    'TCS',
    'TATACONSUM',
    'TATAELXSI',
    'TATAINVEST',
    'TMPV',
    'TATAPOWER',
    'TATASTEEL',
    'TATATECH',
    'TTML',
    'TECHM',
    'TECHNOE',
    'TEJASNET',
    'NIACL',
    'RAMCOCEM',
    'THERMAX',
    'TIMKEN',
    'TITAGARH',
    'TITAN',
    'TORNTPHARM',
    'TORNTPOWER',
    'TARIL',
    'TRENT',
    'TRIDENT',
    'TRIVENI',
    'TRITURBINE',
    'TIINDIA',
    'UCOBANK',
    'UNOMINDA',
    'UPL',
    'UTIAMC',
    'ULTRACEMCO',
    'UNIONBANK',
    'UBL',
    'UNITDSPR',
    'USHAMART',
    'VGUARD',
    'DBREALTY',
    'VTL',
    'VBL',
    'MANYAVAR',
    'VEDL',
    'VENTIVE',
    'VIJAYA',
    'VMM',
    'IDEA',
    'VOLTAS',
    'WAAREEENER',
    'WELCORP',
    'WELSPUNLIV',
    'WHIRLPOOL',
    'WIPRO',
    'WOCKPHARMA',
    'YESBANK',
    'ZFCVINDIA',
    'ZEEL',
    'ZENTEC',
    'ZENSARTECH',
    'ZYDUSLIFE',
    'ECLERX',
]


In [8]:
# retry_tvdatafeed_fetch.py
import time
import random
import pickle
from pathlib import Path

import pandas as pd
from tvDatafeed import TvDatafeed, Interval

# --- CONFIG ---
EXCHANGE = 'NSE'
TIME_FRAME = Interval.in_daily
N_BARS = 2000

# Optional: limit retries per symbol (None = infinite)
MAX_RETRIES_PER_SYMBOL = 10  # or an int like 30

# Backoff settings
BASE_SLEEP = 5          # seconds (initial)
MAX_SLEEP = 300         # seconds (cap)
JITTER = 3              # seconds of random jitter added/subtracted

# Where to save progress (auto-created)
PROGRESS_FILE = Path("all_stock_data.pkl")

# --- Helper functions ---
def save_progress(all_stock_data, path=PROGRESS_FILE):
    """Save progress to disk as a pickle so we can resume."""
    try:
        with open(path, "wb") as f:
            pickle.dump(all_stock_data, f)
    except Exception as e:
        print(f"Warning: failed to save progress to {path}: {e}")

def load_progress(path=PROGRESS_FILE):
    """Load saved progress if present (returns dict)."""
    if path.exists():
        try:
            with open(path, "rb") as f:
                return pickle.load(f)
        except Exception as e:
            print(f"Warning: could not load progress file {path}: {e}")
    return {}

def backoff_sleep(attempt):
    """Always sleep ~3 seconds (fixed delay)."""
    sleep = 3 + random.uniform(-1, 1)   # jitter ±1 sec (optional)
    sleep = max(0, sleep)
    print(f"Sleeping {sleep:.1f}s before retrying...")
    time.sleep(sleep)

# --- Initialize/resume progress ---
all_stock_data = load_progress()
print(f"Loaded progress: {len(all_stock_data)} symbols already fetched.")

# --- Ensure tvDatafeed is initialized (retry loop) ---
tv = None
init_attempt = 0
while tv is None:
    try:
        init_attempt += 1
        print(f"Attempting to initialize TvDatafeed (attempt {init_attempt})...")
        tv = TvDatafeed()  # will raise if credentials/connection problem
        print("TvDatafeed initialized successfully.")
    except Exception as e:
        print(f"Error initializing tvDatafeed: {e}")
        # exponential backoff for initialization
        backoff_sleep(init_attempt)

print(f"Starting data fetch for {len(NIFTY_500_SYMBOLS)} stocks...")

# --- Main fetch loop ---
try:
    for symbol in NIFTY_500_SYMBOLS:
        # skip if already fetched in progress file
        if symbol in all_stock_data:
            print(f"[SKIP] {symbol} already fetched ({len(all_stock_data)} total).")
            continue

        attempt = 0
        while True:
            attempt += 1
            try:
                print(f"[{symbol}] Fetch attempt {attempt}...")
                df = tv.get_hist(symbol=symbol, exchange=EXCHANGE, interval=TIME_FRAME, n_bars=N_BARS)

                # Sometimes tv.get_hist returns None or empty DataFrame; treat that as failure
                if df is None or df.empty:
                    raise ValueError("No data returned (None or empty DataFrame).")

                # Normalize columns (user's earlier standardization)
                df.columns = [col.lower() for col in df.columns]
                df = df.rename(columns={
                    'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'
                })

                # Save into dict and to disk immediately
                all_stock_data[symbol] = df
                save_progress(all_stock_data)
                print(f"[{symbol}] Success — saved ({len(all_stock_data)} total).")
                # small pause to be polite to API
                time.sleep(0.5 + random.random() * 0.5)
                break  # exit retry loop for this symbol

            except KeyboardInterrupt:
                print("\nUser requested abort (KeyboardInterrupt). Exiting loop and saving progress...")
                save_progress(all_stock_data)
                raise

            except Exception as e:
                print(f"[{symbol}] Error on attempt {attempt}: {e}")

                # If max retries is set and reached, break/skip
                if (MAX_RETRIES_PER_SYMBOL is not None) and (attempt >= MAX_RETRIES_PER_SYMBOL):
                    print(f"[{symbol}] Reached max retries ({MAX_RETRIES_PER_SYMBOL}). Skipping symbol.")
                    break

                # If the tv object seems invalid (rare), try reinitializing tv
                if isinstance(e, Exception):
                    # Optionally, detect specific exception types from tvDatafeed to re-init
                    # We'll attempt to recreate tv instance if we see a connection-like problem
                    try:
                        # simple heuristic: if many attempts have failed, reinit tv
                        if attempt % 5 == 0:
                            print("Re-initializing tvDatafeed connection (periodic reinit)...")
                            tv = None
                            init_try = 0
                            while tv is None:
                                init_try += 1
                                try:
                                    tv = TvDatafeed()
                                    print("TvDatafeed re-initialized.")
                                except Exception as ex_init:
                                    print(f"Re-init error: {ex_init}")
                                    backoff_sleep(init_try)

                    except Exception as reinit_e:
                        print(f"Failed during tv reinit attempt: {reinit_e}")

                # Wait with exponential backoff before next attempt
                backoff_sleep(attempt)

    print("\n--- Data Acquisition Complete ---")
    print(f"Total stocks fetched successfully: {len(all_stock_data)}")

except KeyboardInterrupt:
    print("\nScript stopped by user. Progress saved.")
except Exception as final_e:
    print(f"\nFatal error: {final_e}. Progress saved.")
    save_progress(all_stock_data)


Loaded progress: 67 symbols already fetched.
Attempting to initialize TvDatafeed (attempt 1)...
TvDatafeed initialized successfully.
Starting data fetch for 501 stocks...
[SKIP] 360ONE already fetched (67 total).
[SKIP] 3MINDIA already fetched (67 total).
[SKIP] ABB already fetched (67 total).
[SKIP] ACC already fetched (67 total).
[SKIP] ACMESOLAR already fetched (67 total).
[SKIP] AIAENG already fetched (67 total).
[SKIP] APLAPOLLO already fetched (67 total).
[SKIP] AUBANK already fetched (67 total).
[SKIP] AWL already fetched (67 total).
[SKIP] AADHARHFC already fetched (67 total).
[SKIP] AARTIIND already fetched (67 total).
[SKIP] AAVAS already fetched (67 total).
[SKIP] ABBOTINDIA already fetched (67 total).
[SKIP] ACE already fetched (67 total).
[SKIP] ADANIENSOL already fetched (67 total).
[SKIP] ADANIENT already fetched (67 total).
[SKIP] ADANIGREEN already fetched (67 total).
[SKIP] ADANIPORTS already fetched (67 total).
[SKIP] ADANIPOWER already fetched (67 total).
[SKIP] ATG

ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


[BAJAJ-AUTO] Error on attempt 1: No data returned (None or empty DataFrame).
Sleeping 2.7s before retrying...
[BAJAJ-AUTO] Fetch attempt 2...


ERROR:tvDatafeed.main:Connection timed out
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


[BAJAJ-AUTO] Error on attempt 2: No data returned (None or empty DataFrame).
Sleeping 3.8s before retrying...
[BAJAJ-AUTO] Fetch attempt 3...


...

[ZEEL] Error on attempt 3: No data returned (None or empty DataFrame).
Sleeping 2.3s before retrying...
[ZEEL] Fetch attempt 4...


ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


[ZEEL] Error on attempt 4: No data returned (None or empty DataFrame).
Sleeping 2.3s before retrying...
[ZEEL] Fetch attempt 5...


ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


[ZEEL] Error on attempt 5: No data returned (None or empty DataFrame).
Re-initializing tvDatafeed connection (periodic reinit)...
TvDatafeed re-initialized.
Sleeping 3.8s before retrying...
[ZEEL] Fetch attempt 6...
[ZEEL] Success — saved (494 total).
[ZENTEC] Fetch attempt 1...
[ZENTEC] Success — saved (495 total).
[ZENSARTECH] Fetch attempt 1...
[ZENSARTECH] Success — saved (496 total).
[ZYDUSLIFE] Fetch attempt 1...
[ZYDUSLIFE] Success — saved (497 total).
[ECLERX] Fetch attempt 1...
[ECLERX] Success — saved (498 total).

--- Data Acquisition Complete ---
Total stocks fetched successfully: 498


In [9]:
# converting .pkl file to pandas dataframe
# format: columns: OLHC, rows: stock symbols

import pickle
import pandas as pd

# Load the dictionary of DataFrames
with open("all_stock_data.pkl", "rb") as f:
    all_stock_data = pickle.load(f)

# Convert to a multi-index DataFrame
multi_df = pd.concat(all_stock_data, names=["symbol", "datetime"])

print(multi_df)

                                    symbol         Open         High  \
symbol   datetime                                                      
RELIANCE 2017-11-03 09:15:00  NSE:RELIANCE   429.843215   429.843215   
         2017-11-06 09:15:00  NSE:RELIANCE   423.795414   423.795414   
         2017-11-07 09:15:00  NSE:RELIANCE   419.681105   420.423037   
         2017-11-08 09:15:00  NSE:RELIANCE   405.809402   407.383178   
         2017-11-09 09:15:00  NSE:RELIANCE   402.324612   409.136823   
...                                    ...          ...          ...   
ECLERX   2025-11-24 09:15:00    NSE:ECLERX  4463.700000  4554.600000   
         2025-11-25 09:15:00    NSE:ECLERX  4501.600000  4515.100000   
         2025-11-26 09:15:00    NSE:ECLERX  4478.300000  4478.300000   
         2025-11-27 09:15:00    NSE:ECLERX  4459.000000  4512.000000   
         2025-11-28 09:15:00    NSE:ECLERX  4490.400000  4614.000000   

                                      Low        Close        V